This notebook will be used to calculate the baroclinic tide over the CODAR region.

In [6]:
import datetime
import matplotlib.pylab as plt
import numpy as np
import netCDF4 as nc

from salishsea_tools import (viz_tools,tidetools, nc_tools, ellipse)
from salishsea_tools.nowcast import (research_VENUS, analyze)

import seaborn as sns

%matplotlib inline

In [3]:
sns.set_style('darkgrid')

In [4]:
path='/data/dlatorne/MEOPAR/SalishSea/nowcast/'
NodalCorr = tidetools.CorrTides

###Load Data

In [7]:
to=datetime.datetime(2014,11,26)
tf=datetime.datetime(2015,4,26)


fname = '{}_currents_{}_{}.nc'.format('CODAR', to.strftime('%Y%m%d'), tf.strftime('%Y%m%d'))
f = nc.Dataset(fname)
    
us = f.variables['vozocrtx'][:]
vs = f.variables['vomecrty'][:]
depths = f.variables['deptht'][:]
times = f.variables['time_counter'][:]

Unstagger, rotate and calculate baroclinic tide

In [10]:
def nodal_corrections(tide, tidecorr):
    """apply nodal corrections to tidal constituent amplude and phase
    
    tide is a nested dictionary with phase and ampliude for each constituent
    tidecorr is also a nested dictionary with nodal factors and phase shift for each constituent"""
    for const in tide:
        tide[const]['phase'] = (tide[const]['phase'] + tidecorr[const]['uvt'])
        tide[const]['amp'] = tide[const]['amp'] / tidecorr[const]['ft']
        
        shape = tide[const]['phase'].shape
        corr_amp = tide[const]['amp'].flatten()
        corr_phase = tide[const]['phase'].flatten()
        ind = 0
        for amp, phase in zip(corr_amp, corr_phase):
            corr_amp[ind], corr_phase[ind] = tidetools.convention_pha_amp(amp, phase)
            ind = ind +1
        tide[const]['phase'] = np.reshape(corr_phase, shape)
        tide[const]['amp'] = np.reshape(corr_amp, shape)
    return tide

In [11]:
def baroclinic_tide(u, time, depth, nconsts):
    """Perform a harmonic analysis on the baroclinic tide
    
    u is the full depth profile of a current
    If u is from NEMO output, it should be unstaggered before this funcion is applied. 
    u should have at least a time and depth dimension but could also have y, x shape
    the depth dimensions must be in axis 1.
    
    time is the times associated with the current.
    
    depth is an array of depths associated with u
    
    nconsts is the number of constituents to analyze
    
    returns tide_bc - a nested dictionary with amp and phase for each constituent 
    eg. tide_bc['M2']['phase']
    also returns the baroclinic currnt, u_bc"""
    
    #Calculate depth-averaged current
    u_depav = analyze.depth_average(u, depth, depth_axis=1)
    u_depav = np.expand_dims(u_depav, axis=1)
    #Caclulate baroclinic current by removing depth averaged bit
    u_bc = u - u_depav
    
    #tidal analysss of baroclinic current
    tide_bc = tidetools.fittit(u_bc, time, nconst)
    
    return tide_bc, u_bc

In [12]:
#u_tide_bc = {} #baroclinc tidal constituents - u
#v_tide_bc = {} # baroclinic tidal constitnets - v
#u_bc = {} # barocinic current - u
#v_bc = {} # baroclinic current - v
#baroclinic_ellipse = {} # ellipse values for baroclinic tide
u_rot, v_rot = ellipse.prepare_vel(us, vs)
u_tide_bc, u_bc = baroclinic_tide(u_rot, times, depths, nconst)
v_tide_bc, v_bc = baroclinic_tide(v_rot, times, depths, nconst)
#nodal corrections
u_tide_bc = nodal_corrections(u_tide_bc, NodalCorr)
v_tide_bc = nodal_corrections(v_tide_bc, NodalCorr)
baroclinic_ellipse = ellipse.get_params(u_bc, v_bc, times, nconst, tidecorr=NodalCorr)

MemoryError: 